# Import Package

In [30]:
import os 
import pandas as pd
import numpy as np
import sklearn as sl
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

# Import Train dataset

In [31]:
os.chdir(r'C:\Users\Joe\Desktop\Kaggle\Titanic\data')
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [33]:
train.shape

(891, 12)

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [34]:
train_x = train.drop(columns = ['Name','Embarked','Ticket','Cabin','Survived'])
train_y = train['Survived']
test_x = test.drop(columns = ['Name','Embarked','Ticket','Cabin'])

In [35]:
le = preprocessing.LabelEncoder()
train_x['Sex'] = le.fit_transform(train_x['Sex'])
test_x['Sex'] = le.fit_transform(test_x['Sex'])

In [36]:
train_x.isna().sum()

PassengerId      0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

## Handle Age column missing issue

In [37]:
data = [train_x, test_x]
for dataset in data:
    mean = train_x["Age"].mean()
    std = test_x["Age"].std()
    is_null = dataset["Age"].isna().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train_x["Age"].astype(int)

In [38]:
# Handle one fare missing issue by filling in the mean value
test_x.iloc[test_x[test_x['Fare'].isna() == True].index[0],6] = train_x["Fare"].mean()

In [39]:
train_x.isna().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64

In [40]:
train_x.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,3,1,22,1,0,7.2500
1,2,1,0,38,1,0,71.2833
2,3,3,0,26,0,0,7.9250
3,4,1,0,35,1,0,53.1000
4,5,3,1,35,0,0,8.0500


# Fit Dataset (Random Forest)

In [59]:
rf_clf = RandomForestClassifier(n_estimators=10, max_depth = 10)
rf_clf = rf_clf.fit(train_x, train_y)

In [60]:
Y_pred = rf_clf.predict(test_x)

In [61]:
rf_clf.score(train_x, train_y)

0.9775533108866442

In [62]:
Y_pred = pd.DataFrame(data = Y_pred, columns = ['Survived'])

In [63]:
Y_pred['PassengerId'] = test_x['PassengerId']

In [65]:
Y_pred.to_csv(r'rf.csv')